# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4a704e26a0>
├── 'a' --> tensor([[-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621]])
└── 'x' --> <FastTreeValue 0x7f4a704e2670>
    └── 'c' --> tensor([[ 1.2463,  0.2042,  1.5002, -1.6245],
                        [-1.3846, -0.7254,  0.6827,  0.7584],
                        [-1.8846, -0.3727, -0.8035, -0.9024]])

In [4]:
t.a

tensor([[-1.8386,  1.7245, -0.3126],
        [-0.6294,  0.9176,  1.7621]])

In [5]:
%timeit t.a

60.6 ns ± 0.0652 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4a704e26a0>
├── 'a' --> tensor([[ 0.8165, -0.3276, -0.5129],
│                   [-0.1766,  0.3480,  0.8524]])
└── 'x' --> <FastTreeValue 0x7f4a704e2670>
    └── 'c' --> tensor([[ 1.2463,  0.2042,  1.5002, -1.6245],
                        [-1.3846, -0.7254,  0.6827,  0.7584],
                        [-1.8846, -0.3727, -0.8035, -0.9024]])

In [7]:
%timeit t.a = new_value

58.8 ns ± 0.0223 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621]]),
    x: Batch(
           c: tensor([[ 1.2463,  0.2042,  1.5002, -1.6245],
                      [-1.3846, -0.7254,  0.6827,  0.7584],
                      [-1.8846, -0.3727, -0.8035, -0.9024]]),
       ),
)

In [10]:
b.a

tensor([[-1.8386,  1.7245, -0.3126],
        [-0.6294,  0.9176,  1.7621]])

In [11]:
%timeit b.a

57.6 ns ± 0.209 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7545, -1.8119, -1.2611],
               [-1.0523, -1.4646,  0.5181]]),
    x: Batch(
           c: tensor([[ 1.2463,  0.2042,  1.5002, -1.6245],
                      [-1.3846, -0.7254,  0.6827,  0.7584],
                      [-1.8846, -0.3727, -0.8035, -0.9024]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.162 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

858 ns ± 4.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 67.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 934 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4a725733a0>
├── 'a' --> tensor([[[-1.8386,  1.7245, -0.3126],
│                    [-0.6294,  0.9176,  1.7621]],
│           
│                   [[-1.8386,  1.7245, -0.3126],
│                    [-0.6294,  0.9176,  1.7621]],
│           
│                   [[-1.8386,  1.7245, -0.3126],
│                    [-0.6294,  0.9176,  1.7621]],
│           
│                   [[-1.8386,  1.7245, -0.3126],
│                    [-0.6294,  0.9176,  1.7621]],
│           
│                   [[-1.8386,  1.7245, -0.3126],
│                    [-0.6294,  0.9176,  1.7621]],
│           
│                   [[-1.8386,  1.7245, -0.3126],
│                    [-0.6294,  0.9176,  1.7621]],
│           
│                   [[-1.8386,  1.7245, -0.3126],
│                    [-0.6294,  0.9176,  1.7621]],
│           
│                   [[-1.8386,  1.7245, -0.3126],
│                    [-0.6294,  0.9176,  1.7621]]])
└── 'x' --> <FastTreeValue 0x7f49c31edf40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 67.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f49c31c5d60>
├── 'a' --> tensor([[-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621],
│                   [-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621],
│                   [-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621],
│                   [-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621],
│                   [-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621],
│                   [-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621],
│                   [-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621],
│                   [-1.8386,  1.7245, -0.3126],
│                   [-0.6294,  0.9176,  1.7621]])
└── 'x' --> <FastTreeValue 0x7f49c31c58e0>
    └── 'c' --> tensor([[ 1.2463,  0.2042,  1.5002, -1.6245],
                        [-1.3846, -0.7254,  0.6827,  0.7584],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 33 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.8386,  1.7245, -0.3126],
                [-0.6294,  0.9176,  1.7621]],
       
               [[-1.8386,  1.7245, -0.3126],
                [-0.6294,  0.9176,  1.7621]],
       
               [[-1.8386,  1.7245, -0.3126],
                [-0.6294,  0.9176,  1.7621]],
       
               [[-1.8386,  1.7245, -0.3126],
                [-0.6294,  0.9176,  1.7621]],
       
               [[-1.8386,  1.7245, -0.3126],
                [-0.6294,  0.9176,  1.7621]],
       
               [[-1.8386,  1.7245, -0.3126],
                [-0.6294,  0.9176,  1.7621]],
       
               [[-1.8386,  1.7245, -0.3126],
                [-0.6294,  0.9176,  1.7621]],
       
               [[-1.8386,  1.7245, -0.3126],
                [-0.6294,  0.9176,  1.7621]]]),
    x: Batch(
           c: tensor([[[ 1.2463,  0.2042,  1.5002, -1.6245],
                       [-1.3846, -0.7254,  0.6827,  0.7584],
                       [-1.8846, -0.3727, -0.8035, -0.9024]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621],
               [-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621],
               [-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621],
               [-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621],
               [-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621],
               [-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621],
               [-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621],
               [-1.8386,  1.7245, -0.3126],
               [-0.6294,  0.9176,  1.7621]]),
    x: Batch(
           c: tensor([[ 1.2463,  0.2042,  1.5002, -1.6245],
                      [-1.3846, -0.7254,  0.6827,  0.7584],
                      [-1.8846, -0.3727, -0.8035, -0.9024],
                      [ 1.2463,  0.2042,  1.5002, -1.6245],
                      [-1.3846, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 216 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 950 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
